# Automated frequency sweep for acoustofluidic applications - 2023
Author: Fabian Lickert
Date: 02.01.2023

## Import the neccessary classes

In [2]:
from drivers import control_valve as cv
from drivers import control_signal_gen as csg
from drivers import control_pump as cp
from drivers.equalize_voltage import *

import pco
import os # communication with operating system
from skimage import io
import gc
import csv
import time
import matplotlib.pyplot as plt

## Initialize the valve, camera and signal generator controlls

In [ ]:
valve = cv.ControlValve(bdr=19200, port='COM4')
signal = csg.ControlSignalGenNew()
pump = cp.ControlPump(bdr=115200, port='COM5')

signal.set_voltage(0.1)
pump.set_rate(500)

target_voltage = 1
delay_time = 8
exposure_time = 2
exposure_time_ns = 1E6*exposure_time
fps_mhz = 25*1E3


## Perform a sweep in frequency

In [ ]:
try:

    with pco.Camera() as cam:

        cam.configuration = {'timestamp': 'binary & ascii'}
        cam.sdk.set_delay_exposure_time(delay_time, 'ms', exposure_time, 'ms')
        #cam.sdk.set_frame_rate(fps_mhz, exposure_time_ns)
        cam.sdk.set_roi(1, 100, 2560, 1960)
        #cam.sdk.set_trigger_mode('auto sequence')
        #cam.sdk.set_hwio_signal(3, 'on', 'TTL', 'high level', 'off', 4, [0, 0, 0, 0])
        #print(cam.sdk.get_hwio_signal(3))
           
        for f in range(1000000, 1100000, 10000):

            print(f"Recording movie at frequency f = {f}")
            filename = f"mov_{f}_img_no_"
            
            # set current frequency and start the signal generator
            signal.set_freq(f)
            
            # stabilize voltage at 1 V
            signal.reset_trigger()
            
            voltage_list = equalize_voltage(signal, 0.1, target_voltage, f)
            
            final_voltage = voltage_list[0]
            set_voltage = voltage_list[1]
            
            print(f"Voltage stabilized at: {final_voltage}")

            logging = True
            if logging:
                with open('voltage_log_WNB.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([f"{f}", f"{final_voltage:.4}", f"{set_voltage:.4}"])  

            # set valve to position 1 for x seconds to flush the channel at defined rate
            print("Flushing the channel")
            valve.set_pos(1)
            pump.run()
            time.sleep(10)
            pump.stop()
            print("Flushing finished.")
            
            # Set valve to position 2 to stop the flow and stabilize for 10 seconds
            time.sleep(10)
            valve.set_pos(2)
            time.sleep(10)
            
            signal.set_trigger(1, 0, 1)
            signal.run()

            # record 200 images
            cam.record(number_of_images=200, mode='sequence')
            images, metadatas = cam.images()

            time.sleep(2)
            signal.stop()

            path = f"img/freq_{f}/"
            print(path)
            try:
                os.mkdir(path)
            except OSError as error:
                print(error)    
            
            print("Saving the images...")
            name = "image"
            count = 1
            
            for image in images:
                io.imsave(path + name+"{0:04d}.tif".format(count), image, check_contrast=False)
                count += 1
            
            print("Images saved!")

            del image
            del images
            del metadatas
            
            gc.collect()
            
except:

    print("Found exception... aborting...")
    valve.close()
    signal.close()
    pump.close()


## Close connections to all the instruments again

In [ ]:
valve.close()
signal.close()
pump.close()